# Garbage Classification

### Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal as signalpy
import tensorflow as tf
from PIL import Image
import keras_tuner as kt
import kagglehub
import shutil
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import random

### Useful Functions

In [2]:
def count_images_in_folder(folder_path, image_extensions=(".jpg", ".jpeg", ".png", ".gif", ".bmp")):
    count = 0
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(image_extensions):  # Check for valid image extensions
            count += 1
    return count

def count_jpg_images_in_folder(folder_path, image_extensions=(".jpg")):
    count = 0
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(image_extensions):  # Check for valid image extensions
            count += 1
    return count

def load_image_and_label(image_path , label) :
    image_string = tf.io.read_file(image_path)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_decoded.set_shape([224, 224, 3])   
    return image_decoded, label

## Data Preprocessing

### Download the datasets and move them to the repo folder

In [3]:
# UNCOMMENT WHEN SETTING THE DATASET UP FOR THE FIRST TIME

# kaggle_default_dataset_1_path = kagglehub.dataset_download("farzadnekouei/trash-type-image-dataset")
# print("Path to dataset 1 files :", kaggle_default_dataset_1_path)
# kaggle_default_dataset_2_path = kagglehub.dataset_download("mostafaabla/garbage-classification")
# print("Path to dataset 2 files :", kaggle_default_dataset_2_path)

In [4]:
# UNCOMMENT WHEN SETTING THE DATASET UP FOR THE FIRST TIME

# kaggle_default_dataset_1_waste_categories_path = os.path.join(kaggle_default_dataset_1_path, 'TrashType_Image_Dataset')
# kaggle_default_dataset_2_waste_categories_path = os.path.join(kaggle_default_dataset_2_path, 'garbage_classification')

In [5]:
dataset_parent_folder_path = os.path.join(os.getcwd(),'Dataset')
dataset_1_path = os.path.join(dataset_parent_folder_path, 'Dataset 1')
dataset_2_path = os.path.join(dataset_parent_folder_path, 'Dataset 2')
os.makedirs(dataset_parent_folder_path, exist_ok=True)
os.makedirs(dataset_1_path, exist_ok=True)
os.makedirs(dataset_2_path, exist_ok=True)

In [6]:
# UNCOMMENT WHEN SETTING THE DATASET UP FOR THE FIRST TIME

# shutil.copytree(kaggle_default_dataset_1_waste_categories_path, dataset_1_path, dirs_exist_ok=True)
# shutil.rmtree(kaggle_default_dataset_1_path)
# shutil.copytree(kaggle_default_dataset_2_waste_categories_path, dataset_2_path, dirs_exist_ok=True)
# shutil.rmtree(kaggle_default_dataset_2_path)

### Give a generic structure to the dataset

Waste Types Folders :
1. general_waste
2. paper_waste
3. plastic_waste
4. metal_waste
5. textile_waste
6. electronic_waste
7. glass_waste

#### Dataset 1 modifications

In [7]:
# UNCOMMENT WHEN SETTING THE DATASET UP FOR THE FIRST TIME

# if os.path.exists(os.path.join(dataset_1_path, 'glass')) and not os.path.exists(os.path.join(dataset_1_path, 'glass_waste')) :
#     os.replace(os.path.join(dataset_1_path, 'glass'), os.path.join(dataset_1_path, 'glass_waste'))
# if os.path.exists(os.path.join(dataset_1_path, 'metal')) and not os.path.exists(os.path.join(dataset_1_path, 'metal_waste')) :
#     os.rename(os.path.join(dataset_1_path, 'metal'), os.path.join(dataset_1_path, 'metal_waste'))
# if os.path.exists(os.path.join(dataset_1_path, 'paper')) and not os.path.exists(os.path.join(dataset_1_path, 'paper_waste')) :
#     os.rename(os.path.join(dataset_1_path, 'paper'), os.path.join(dataset_1_path, 'paper_waste'))
# if os.path.exists(os.path.join(dataset_1_path, 'plastic')) and not os.path.exists(os.path.join(dataset_1_path, 'plastic_waste')) :
#     os.rename(os.path.join(dataset_1_path, 'plastic'), os.path.join(dataset_1_path, 'plastic_waste'))
# if os.path.exists(os.path.join(dataset_1_path, 'trash')) and not os.path.exists(os.path.join(dataset_1_path, 'general_waste')) :
#     os.rename(os.path.join(dataset_1_path, 'trash'), os.path.join(dataset_1_path, 'general_waste'))

# if os.path.exists(os.path.join(dataset_1_path, 'cardboard')) :
#     shutil.copytree(os.path.join(dataset_1_path, 'cardboard'), os.path.join(dataset_1_path, 'paper_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_1_path, 'cardboard'))

dataset_1_general_waste_path = os.path.join(dataset_1_path, 'general_waste')
dataset_1_glass_waste_path = os.path.join(dataset_1_path, 'glass_waste')
dataset_1_metal_waste_path = os.path.join(dataset_1_path, 'metal_waste')
dataset_1_paper_waste_path = os.path.join(dataset_1_path, 'paper_waste')
dataset_1_plastic_waste_path = os.path.join(dataset_1_path, 'plastic_waste')

#### Dataset 2 modifications

In [8]:
# UNCOMMENT WHEN SETTING THE DATASET UP FOR THE FIRST TIME

# if os.path.exists(os.path.join(dataset_2_path, 'battery')) and not os.path.exists(os.path.join(dataset_2_path, 'electronic_waste')) :
#     os.replace(os.path.join(dataset_2_path, 'battery'), os.path.join(dataset_2_path, 'electronic_waste'))

# if os.path.exists(os.path.join(dataset_2_path, 'trash')) and not os.path.exists(os.path.join(dataset_2_path, 'general_waste')) :
#     os.replace(os.path.join(dataset_2_path, 'trash'), os.path.join(dataset_2_path, 'general_waste'))

# if os.path.exists(os.path.join(dataset_2_path, 'metal')) and not os.path.exists(os.path.join(dataset_2_path, 'metal_waste')) :
#     os.replace(os.path.join(dataset_2_path, 'metal'), os.path.join(dataset_2_path, 'metal_waste'))

# if os.path.exists(os.path.join(dataset_2_path, 'paper')) and not os.path.exists(os.path.join(dataset_2_path, 'paper_waste')) :
#     os.replace(os.path.join(dataset_2_path, 'paper'), os.path.join(dataset_2_path, 'paper_waste'))

# if os.path.exists(os.path.join(dataset_2_path, 'plastic')) and not os.path.exists(os.path.join(dataset_2_path, 'plastic_waste')) :
#     os.replace(os.path.join(dataset_2_path, 'plastic'), os.path.join(dataset_2_path, 'plastic_waste'))

# os.makedirs(os.path.join(dataset_2_path, 'glass_waste'), exist_ok=True)
# os.makedirs(os.path.join(dataset_2_path, 'textile_waste'), exist_ok=True)

# if os.path.exists(os.path.join(dataset_2_path, 'biological')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'biological'), os.path.join(dataset_2_path, 'general_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'biological'))

# if os.path.exists(os.path.join(dataset_2_path, 'brown-glass')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'brown-glass'), os.path.join(dataset_2_path, 'glass_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'brown-glass'))

# if os.path.exists(os.path.join(dataset_2_path, 'green-glass')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'green-glass'), os.path.join(dataset_2_path, 'glass_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'green-glass'))

# if os.path.exists(os.path.join(dataset_2_path, 'white-glass')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'white-glass'), os.path.join(dataset_2_path, 'glass_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'white-glass'))

# if os.path.exists(os.path.join(dataset_2_path, 'shoes')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'shoes'), os.path.join(dataset_2_path, 'textile_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'shoes'))

# if os.path.exists(os.path.join(dataset_2_path, 'clothes')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'clothes'), os.path.join(dataset_2_path, 'textile_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'clothes'))

# if os.path.exists(os.path.join(dataset_2_path, 'cardboard')) :
#     shutil.copytree(os.path.join(dataset_2_path, 'cardboard'), os.path.join(dataset_2_path, 'paper_waste'), dirs_exist_ok=True)
#     shutil.rmtree(os.path.join(dataset_2_path, 'cardboard'))

dataset_2_electronic_waste_path = os.path.join(dataset_2_path, 'electronic_waste')
dataset_2_general_waste_path = os.path.join(dataset_2_path, 'general_waste')
dataset_2_glass_waste_path = os.path.join(dataset_2_path, 'glass_waste')
dataset_2_metal_waste_path = os.path.join(dataset_2_path, 'metal_waste')
dataset_2_paper_waste_path = os.path.join(dataset_2_path, 'paper_waste')
dataset_2_plastic_waste_path = os.path.join(dataset_2_path, 'plastic_waste')
dataset_2_textile_waste_path = os.path.join(dataset_2_path, 'textile_waste', 'refined_textile_waste')

#### Final Dataset modifications

In [9]:
final_dataset_path = os.path.join(dataset_parent_folder_path, "Final_Dataset")
final_dataset_general_waste_path = os.path.join(final_dataset_path, "general_waste")
final_dataset_paper_waste_path = os.path.join(final_dataset_path, "paper_waste")
final_dataset_plastic_waste_path = os.path.join(final_dataset_path, "plastic_waste")
final_dataset_metal_waste_path = os.path.join(final_dataset_path, "metal_waste")
final_dataset_unselected_textile_waste_path = os.path.join(final_dataset_path, "textile_waste", 'refined_textile_waste')
final_dataset_textile_waste_path = os.path.join(final_dataset_unselected_textile_waste_path, 'selected_refined_textile_waste')
final_dataset_electronic_waste_path = os.path.join(final_dataset_path, "electronic_waste")
final_dataset_glass_waste_path = os.path.join(final_dataset_path, "glass_waste")
os.makedirs(final_dataset_path, exist_ok=True)
os.makedirs(final_dataset_general_waste_path, exist_ok=True)
os.makedirs(final_dataset_paper_waste_path, exist_ok=True)
os.makedirs(final_dataset_plastic_waste_path, exist_ok=True)
os.makedirs(final_dataset_metal_waste_path, exist_ok=True)
os.makedirs(final_dataset_unselected_textile_waste_path, exist_ok=True)
os.makedirs(final_dataset_textile_waste_path, exist_ok=True)
os.makedirs(final_dataset_electronic_waste_path, exist_ok=True)
os.makedirs(final_dataset_glass_waste_path, exist_ok=True)

# UNCOMMENT WHEN SETTING THE DATASET UP FOR THE FIRST TIME

# clothes_images = [f for f in os.listdir(final_dataset_unselected_textile_waste_path) if f.lower().startswith("clothes")]
# shoes_images = [f for f in os.listdir(final_dataset_unselected_textile_waste_path) if f.lower().startswith("shoes")]

# random.shuffle(clothes_images)
# random.shuffle(shoes_images)

# num_images = 1000
# selected_clothes_images = clothes_images[:num_images]
# selected_shoes_images = shoes_images[:num_images]

# for image in selected_clothes_images :
#     shutil.copy2(os.path.join(final_dataset_unselected_textile_waste_path, image), os.path.join(final_dataset_textile_waste_path, image))
# for image in selected_shoes_images :
#     shutil.copy2(os.path.join(final_dataset_unselected_textile_waste_path, image), os.path.join(final_dataset_textile_waste_path, image))

# if os.path.exists(final_dataset_electronic_waste_path) and os.path.exists(dataset_2_electronic_waste_path) :
#     for item in os.listdir(final_dataset_electronic_waste_path) :
#         item_path = os.path.join(final_dataset_electronic_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_2_electronic_waste_path, final_dataset_electronic_waste_path, dirs_exist_ok=True)

# if os.path.exists(final_dataset_general_waste_path) and os.path.exists(dataset_1_general_waste_path) and os.path.exists(dataset_2_general_waste_path) :
#     for item in os.listdir(final_dataset_general_waste_path) :
#         item_path = os.path.join(final_dataset_general_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_1_general_waste_path, final_dataset_general_waste_path, dirs_exist_ok=True)
#     shutil.copytree(dataset_2_general_waste_path, final_dataset_general_waste_path, dirs_exist_ok=True)

# if os.path.exists(final_dataset_glass_waste_path) and os.path.exists(dataset_1_glass_waste_path) and os.path.exists(dataset_2_glass_waste_path) :
#     for item in os.listdir(final_dataset_glass_waste_path) :
#         item_path = os.path.join(final_dataset_glass_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_1_glass_waste_path, final_dataset_glass_waste_path, dirs_exist_ok=True)
#     shutil.copytree(dataset_2_glass_waste_path, final_dataset_glass_waste_path, dirs_exist_ok=True)

# if os.path.exists(final_dataset_metal_waste_path) and os.path.exists(dataset_1_metal_waste_path) and os.path.exists(dataset_2_metal_waste_path) :
#     for item in os.listdir(final_dataset_metal_waste_path) :
#         item_path = os.path.join(final_dataset_metal_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_1_metal_waste_path, final_dataset_metal_waste_path, dirs_exist_ok=True)
#     shutil.copytree(dataset_2_metal_waste_path, final_dataset_metal_waste_path, dirs_exist_ok=True)

# if os.path.exists(final_dataset_paper_waste_path) and os.path.exists(dataset_1_paper_waste_path) and os.path.exists(dataset_2_paper_waste_path) :
#     for item in os.listdir(final_dataset_paper_waste_path) :
#         item_path = os.path.join(final_dataset_paper_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_1_paper_waste_path, final_dataset_paper_waste_path, dirs_exist_ok=True)
#     shutil.copytree(dataset_2_paper_waste_path, final_dataset_paper_waste_path, dirs_exist_ok=True)

# if os.path.exists(final_dataset_plastic_waste_path) and os.path.exists(dataset_1_plastic_waste_path) and os.path.exists(dataset_2_plastic_waste_path) :
#     for item in os.listdir(final_dataset_plastic_waste_path) :
#         item_path = os.path.join(final_dataset_plastic_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_1_plastic_waste_path, final_dataset_plastic_waste_path, dirs_exist_ok=True)
#     shutil.copytree(dataset_2_plastic_waste_path, final_dataset_plastic_waste_path, dirs_exist_ok=True)

# if os.path.exists(final_dataset_textile_waste_path) and os.path.exists(dataset_2_textile_waste_path) :
#     for item in os.listdir(final_dataset_textile_waste_path) :
#         item_path = os.path.join(final_dataset_textile_waste_path, item)
#         if os.path.isfile(item_path) or os.path.islink(item_path) :
#             os.unlink(item_path)
#         elif os.path.isdir(item_path) :
#             shutil.rmtree(item_path)
#     shutil.copytree(dataset_2_textile_waste_path, final_dataset_textile_waste_path, dirs_exist_ok=True)

#### Waste types data comparison

In [ ]:
waste_types = ["Electronic", "General", "Paper", "Glass", "Plastic", "Metal", "Textile"]
electronic_waste_image_count = count_images_in_folder(final_dataset_electronic_waste_path)
general_waste_image_count = count_images_in_folder(final_dataset_general_waste_path)
paper_waste_image_count = count_images_in_folder(final_dataset_paper_waste_path)
glass_waste_image_count = count_images_in_folder(final_dataset_glass_waste_path)
plastic_waste_image_count = count_images_in_folder(final_dataset_plastic_waste_path)
metal_waste_image_count = count_images_in_folder(final_dataset_metal_waste_path)
textile_waste_image_count = count_images_in_folder(final_dataset_textile_waste_path)

image_counts = [electronic_waste_image_count, general_waste_image_count, paper_waste_image_count, glass_waste_image_count, plastic_waste_image_count, metal_waste_image_count, textile_waste_image_count]

plt.figure(figsize=(10,6))
bars = plt.bar(waste_types, image_counts)

for bar in bars:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 5,
             str(bar.get_height()), ha='center', fontsize=10)
    
# Adding labels and title
plt.xlabel("Waste Types")
plt.ylabel("Number of Images")
plt.title("Histogram of Image Counts by Waste Type")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display the plot
plt.tight_layout()
plt.show()

### Prepare the data pipeline using tf.keras.utils.image_dataset_from_directory

In [ ]:
waste_classification_dataset = tf.keras.utils.image_dataset_from_directory(
    final_dataset_path,
    labels='inferred',
    label_mode='categorical',  # For multi-class classification
    batch_size=32,
    image_size=(224, 224),  # ResNet50 input size
    shuffle=True,
    seed=123
)

class_names = waste_classification_dataset.class_names

for images, labels in waste_classification_dataset.take(1):  # Take one batch only
    plt.figure(figsize=(12, 12))
    for i in range(9):  # Show the first 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert tensor to image format
        plt.title(f"Label: {class_names[labels[i].numpy().argmax()]}")  # Display the class index
        plt.axis("off")
    plt.show()


# waste_classification_dataset = waste_classification_dataset.shuffle(len(waste_classification_dataset), seed=100, reshuffle_each_iteration=False)
# waste_classification_dataset = waste_classification_dataset.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)

### Splitting the dataset

In [12]:
training_dataset_size = int(0.7*len(waste_classification_dataset))
validation_dataset_size = int(0.15*len(waste_classification_dataset))
testing_dataset_size = len(waste_classification_dataset) - training_dataset_size - validation_dataset_size

training_dataset = waste_classification_dataset.take(training_dataset_size)
validation_dataset = waste_classification_dataset.skip(training_dataset_size).take(validation_dataset_size)
testing_dataset = waste_classification_dataset.skip(training_dataset_size+validation_dataset_size).take(testing_dataset_size)

#### Check the first 9 images from the training dataset

In [ ]:
for images, labels in training_dataset.take(1):  # Take one batch only
    plt.figure(figsize=(12, 12))
    for i in range(9):  # Show the first 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert tensor to image format
        plt.title(f"Label: {class_names[labels[i].numpy().argmax()]}")  # Display the class index
        plt.axis("off")
    plt.show()

#### Check the first 9 images from the validation dataset

In [ ]:
for images, labels in validation_dataset.take(1):  # Take one batch only
    plt.figure(figsize=(12, 12))
    for i in range(9):  # Show the first 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert tensor to image format
        plt.title(f"Label: {class_names[labels[i].numpy().argmax()]}")  # Display the class index
        plt.axis("off")
    plt.show()

#### Check the first 9 images from the testing dataset

In [ ]:
for images, labels in testing_dataset.take(1):  # Take one batch only
    plt.figure(figsize=(12, 12))
    for i in range(9):  # Show the first 9 images
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # Convert tensor to image format
        plt.title(f"Label: {class_names[labels[i].numpy().argmax()]}")  # Display the class index
        plt.axis("off")
    plt.show()

#### Configure the datasets for performance

In [16]:
training_dataset = training_dataset.cache()
validation_dataset = validation_dataset.cache()
testing_dataset = testing_dataset.cache()
training_dataset = training_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)
testing_dataset = testing_dataset.prefetch(tf.data.AUTOTUNE)

#### Preprocess the datasets for ResNet50

In [17]:
training_dataset = training_dataset.map(lambda x, y: (tf.keras.applications.resnet50.preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (tf.keras.applications.resnet50.preprocess_input(x), y))
testing_dataset = testing_dataset.map(lambda x, y: (tf.keras.applications.resnet50.preprocess_input(x), y))

In [ ]:
# Number of batches
print(len(training_dataset))
print(len(validation_dataset))
print(len(testing_dataset))

### Create ResNet base model for feature extraction ignoring the "top" layers used for classification of the ImageNet dataset

In [ ]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers
base_model.summary()

##### Add a new classification head for our waste classification problem

In [ ]:
waste_classification_resnet50_model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')  # 7 classes
])
waste_classification_resnet50_model.summary()

In [22]:
waste_classification_resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
waste_classification_resnet50_model_history = waste_classification_resnet50_model.fit(training_dataset, epochs=10, validation_data=validation_dataset)

##### Evaluate the ResNet50 model on the testing dataset 

In [ ]:
test_loss, test_accuracy = waste_classification_resnet50_model.evaluate(testing_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
for images, labels in testing_dataset.take(1):
    predictions = waste_classification_resnet50_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels.numpy(), axis=1)

    # Visualize some images with their predicted labels
    plt.figure(figsize=(12, 12))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Pred: {class_names[predicted_classes[i]]}\nTrue: {class_names[true_classes[i]]}")
        plt.axis("off")
    plt.show()

In [ ]:
testing_dataset = testing_dataset.unbatch()
y_true, y_pred = [], []
for images, labels in testing_dataset:
    predictions = waste_classification_resnet50_model.predict(tf.expand_dims(images, axis=0))
    predicted_class = np.argmax(predictions, axis=1)[0]
    true_class = np.argmax(labels.numpy())
    y_pred.append(predicted_class)
    y_true.append(true_class)

# Convert to NumPy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
plt.savefig("resnet50_confusion_matrix.png", dpi=300)

# Classification report
classification_report_str = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:\n", classification_report_str)
with open("resnet50_waste_classification_report.txt", "w") as file:
    file.write(classification_report_str)

In [31]:
waste_classification_resnet50_model.save('waste_classification_model_with_resnet50_as_basemodel.keras')

#### Fine-tune also the top layers of the ResNet50 base model for our waste classification dataset

In [ ]:
base_model.trainable = True
print('Number of layers in the base model : ', len(base_model.layers))
fine_tune_at = 125 # Just an example. Adjust as needed. ResNet50 base model has 175 layers
for layer in base_model.layers[:fine_tune_at] :
    layer.trainable = False
waste_classification_resnet50_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
waste_classification_resnet50_model.summary()

In [ ]:
fine_tune_epochs = 5
initial_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs
fine_tuned_waste_classification_resnet50_model_history = waste_classification_resnet50_model.fit(training_dataset, validation_data=validation_dataset, epochs=total_epochs, initial_epoch=len(waste_classification_resnet50_model_history.epoch))

##### Evaluate the finetuned ResNet50 model on the testing dataset 

In [ ]:
finetuned_test_loss, finetuned_test_accuracy = waste_classification_resnet50_model.evaluate(testing_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
for images, labels in testing_dataset.take(1):
    predictions = waste_classification_resnet50_model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels.numpy(), axis=1)

    # Visualize some images with their predicted labels
    plt.figure(figsize=(12, 12))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Pred: {class_names[predicted_classes[i]]}\nTrue: {class_names[true_classes[i]]}")
        plt.axis("off")
    plt.show()

In [ ]:
# testing_dataset = testing_dataset.unbatch()
y_true, y_pred = [], []
for images, labels in testing_dataset:
    predictions = waste_classification_resnet50_model.predict(tf.expand_dims(images, axis=0))
    predicted_class = np.argmax(predictions, axis=1)[0]
    true_class = np.argmax(labels.numpy())
    y_pred.append(predicted_class)
    y_true.append(true_class)

# Convert to NumPy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

plt.savefig("finetuned_resnet50_confusion_matrix.png", dpi=300)

# Classification report
classification_report_str_finetuned_model = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:\n", classification_report_str_finetuned_model)
with open("finetuned_resnet50_waste_classification_report.txt", "w") as file:
    file.write(classification_report_str_finetuned_model)

In [ ]:
waste_classification_resnet50_model.save('waste_classification_finetuned_model_with_resnet50_as_basemodel.keras')